In [1]:
# src/test_storage.py
from src.utils.data_loader import DataLoader
from src.storage.data_store import DataStore
from src.utils.schemas import SCHEMAS

def main():
    loader = DataLoader(SCHEMAS)
    cleaned = loader.load_all("data")
    genres = cleaned.get("genres", [])
    print(f"Loaded {len(genres)} genres.")

    # Use reuse_free_slots=True to reuse deleted IDs (set False to always append)
    store = DataStore(index_attributes=["id", "name"], reuse_free_slots=True)

    # Insert all genres
    for record in genres:
        store.insert_record(record)
    print("Inserted into DataStore.")

    print("\n=== Search by name 'strategy' ===")
    results = store.search_by_attr("name", "strategy")
    print(results)

    print("\n=== Genres with id between 10 and 20 ===")
    ranged = store.range_query("id", 10, 20)
    print("Found:", len(ranged))
    for r in ranged[:10]:
        print(r)

    print("\n=== Delete record 0 ===")
    ok = store.delete_record(0)
    print("Deleted:", ok)
    print("Record 0 now:", store.get_record(0))

    new_id = store.insert_record(genres[0])
    print("Reinserted at ID:", new_id)
    print("Record at reinserted ID:", store.get_record(new_id))
    print("=== End of DataStore test ===")

if __name__ == "__main__":
    main()



Loaded 154 genres.
Inserted into DataStore.

=== Search by name 'strategy' ===
[{'id': 5, 'name': 'strategy'}]

=== Genres with id between 10 and 20 ===
Found: 11
{'id': 10, 'name': 'simulaatio'}
{'id': 11, 'name': 'simulation'}
{'id': 12, 'name': 'sexual content'}
{'id': 13, 'name': 'azione'}
{'id': 14, 'name': 'aksiyon'}
{'id': 15, 'name': 'racing'}
{'id': 16, 'name': 'massively multiplayer'}
{'id': 17, 'name': 'simuladores'}
{'id': 18, 'name': 'eventyr'}
{'id': 19, 'name': 'mmo'}

=== Delete record 0 ===
Deleted: True
Record 0 now: None
Reinserted at ID: 0
Record at reinserted ID: {'id': 1, 'name': 'aventură'}
=== End of DataStore test ===


In [3]:
# src/test_storage.py

from src.utils.data_loader import DataLoader
from src.storage.data_store import DataStore
from src.utils.schemas import SCHEMAS
from src.query_engine.query_handler import QueryEngine


def main():
    # -----------------------------
    # 1) Load data using DataLoader
    # -----------------------------
    loader = DataLoader(SCHEMAS)
    cleaned = loader.load_all("data")
    genres = cleaned.get("genres", [])
    print(f"Loaded {len(genres)} genres.")

    # --------------------------------------
    # 2) Create DataStore and insert records
    # --------------------------------------
    # Use reuse_free_slots=True to reuse deleted IDs (set False to always append)
    store = DataStore(index_attributes=["id", "name"], reuse_free_slots=True)

    # Insert all genres into DataStore
    for record in genres:
        store.insert_record(record)

    print("Inserted into DataStore.\n")

    # --------------------------------------
    # 3) DataStore tests
    # --------------------------------------
    print("========== DATASTORE TESTS ==========\n")

    # Search by name using AVL index
    print("=== DataStore: Search by name 'strategy' ===")
    results = store.search_by_attr("name", "strategy")
    print(results)

    # Range query on id
    print("\n=== DataStore: Genres with id between 10 and 20 ===")
    ranged = store.range_query("id", 10, 25)
    print("Found:", len(ranged))
    for r in ranged[:10]:
        print(r)

    # Delete by internal record ID
    print("\n=== DataStore: Delete record with internal ID 0 ===")
    ok = store.delete_record(0)
    print("Deleted:", ok)
    print("Record 0 now:", store.get_record(0))

    # Reinsert one record and check ID reuse
    new_id = store.insert_record(genres[0])
    print("Reinserted at internal ID:", new_id)
    print("Record at reinserted ID:", store.get_record(new_id))

    print("\n=== End of DataStore tests ===\n")

    # --------------------------------------
    # 4) QueryEngine tests
    # --------------------------------------
    print("========== QUERY ENGINE TESTS ==========\n")

    # Create a QueryEngine on top of the same DataStore.
    engine = QueryEngine(store, key_attribute="id")

    # Pick a target record from the original genres list
    if not genres:
        print("No genres loaded, skipping QueryEngine tests.")
        return

    target_record = genres[10] if len(genres) > 10 else genres[0]
    target_id = target_record["id"]
    original_name = target_record["name"]

    print(f"Target record for tests: id={target_id}, name='{original_name}'")

    # ---- QueryEngine: search_record (READ) ----
    print("\n=== QueryEngine: search_record(id) ===")
    qe_search_results = engine.search_record(target_id)
    print(qe_search_results)

    # ---- QueryEngine: update_record (UPDATE) ----
    print("\n=== QueryEngine: update_record(id, 'name', 'test_name') ===")
    updated = engine.update_record(target_id, "name", "test_name")
    print("Updated:", updated)
    print("After update:", engine.search_record(target_id))

    # Restore original name so we don't leave dirty state
    engine.update_record(target_id, "name", original_name)
    print("Restored original name:", engine.search_record(target_id))

    # ---- QueryEngine: range_query (delegated) ----
    print("\n=== QueryEngine: range_query('id', 10, 20) ===")
    qe_range_results = engine.range_query("id", 10, 20)
    print(f"Found: {len(qe_range_results)} records")
    for rec in qe_range_results[:10]:
        print(rec)

    # ---- QueryEngine: delete_record (DELETE) ----
    print(f"\n=== QueryEngine: delete_record(id={target_id}) ===")
    deleted = engine.delete_record(target_id)
    print("Deleted:", deleted)
    print("After delete, search_record(id):", engine.search_record(target_id))

    print("\n=== End of QueryEngine tests ===")


if __name__ == "__main__":
    main()


Loaded 154 genres.
Inserted into DataStore.

========== DATASTORE TESTS ==========

=== DataStore: Search by name 'strategy' ===
[{'id': 5, 'name': 'strategy'}]

=== DataStore: Genres with id between 10 and 20 ===
Found: 16
{'id': 10, 'name': 'simulaatio'}
{'id': 11, 'name': 'simulation'}
{'id': 12, 'name': 'sexual content'}
{'id': 13, 'name': 'azione'}
{'id': 14, 'name': 'aksiyon'}
{'id': 15, 'name': 'racing'}
{'id': 16, 'name': 'massively multiplayer'}
{'id': 17, 'name': 'simuladores'}
{'id': 18, 'name': 'eventyr'}
{'id': 19, 'name': 'mmo'}

=== DataStore: Delete record with internal ID 0 ===
Deleted: True
Record 0 now: None
Reinserted at internal ID: 0
Record at reinserted ID: {'id': 1, 'name': 'aventură'}

=== End of DataStore tests ===

========== QUERY ENGINE TESTS ==========

Target record for tests: id=11, name='simulation'

=== QueryEngine: search_record(id) ===
[{'id': 11, 'name': 'simulation'}]

=== QueryEngine: update_record(id, 'name', 'test_name') ===
Updated: True
After 